##gDrive mount & setting dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Import

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchsummary

from torch.optim.adam import Adam
from torch.utils.data.dataloader import Dataset , DataLoader

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob

##Set device

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


##Prepare Dataset

In [ ]:
!mkdir "/content/folder"

In [ ]:
%cd "/content/folder"

/content/folder


In [ ]:
!pwd

/content/folder


In [ ]:
!unzip -qq "/content/drive/MyDrive/hackertone/waveone_train.zip" -d "/content/folder/"
!unzip -qq "/content/drive/MyDrive/hackertone/waveone_test.zip" -d "/content/folder/"
!unzip -qq "/content/drive/MyDrive/hackertone/3_train.zip" -d "/content/folder/"

##Setting data root

In [ ]:
train_list = glob.glob('/content/folder/waveone_train/train/*/*') # (* : 모든 파일 선택), 모든 class에 있는 모든 사진 주소를 받아옴
test_list = glob.glob('/content/folder/test/*/*') #test1폴더에 있는 모든 파일(사진)주소

#print(train_img_list) #앞에 주석 제거하시면 어떻게 받아진건지 확인 할 수 있습니다.
# print(test_img_list)

In [ ]:
trainSet = pd.DataFrame(columns=['train_path', 'label']) #
trainSet['train_path'] = train_list
trainSet['label'] = trainSet['train_path'].apply(lambda x : str(x).split('/')[-2])

for i in range(len(trainSet['label'])):
    if trainSet['label'][i] == '1':
        trainSet['label'][i] = np.array([1,0,0])
    elif trainSet['label'][i] == '2':
        trainSet['label'][i] = np.array([0,1,0])
    elif trainSet['label'][i] == '3':
        trainSet['label'][i] = np.array([0,0,1])


print(trainSet)

                                         train_path      label
0    /content/folder/waveone_train/train/3/3_72.csv  [0, 0, 1]
1    /content/folder/waveone_train/train/3/3_85.csv  [0, 0, 1]
2    /content/folder/waveone_train/train/3/3_14.csv  [0, 0, 1]
3    /content/folder/waveone_train/train/3/3_91.csv  [0, 0, 1]
4    /content/folder/waveone_train/train/3/3_28.csv  [0, 0, 1]
..                                              ...        ...
295  /content/folder/waveone_train/train/2/2_96.csv  [0, 1, 0]
296  /content/folder/waveone_train/train/2/2_99.csv  [0, 1, 0]
297  /content/folder/waveone_train/train/2/2_56.csv  [0, 1, 0]
298  /content/folder/waveone_train/train/2/2_61.csv  [0, 1, 0]
299  /content/folder/waveone_train/train/2/2_39.csv  [0, 1, 0]

[300 rows x 2 columns]


In [ ]:
testSet = pd.DataFrame(columns=['test_path', 'label']) #
testSet['test_path'] = test_list
testSet['label'] = testSet['test_path'].apply(lambda x : str(x).split('/')[-2])

for i in range(len(testSet['label'])):
    if testSet['label'][i] == '1':
        testSet['label'][i] = np.array([1,0,0])
    elif testSet['label'][i] == '2':
        testSet['label'][i] = np.array([0,1,0])
    elif testSet['label'][i] == '3':
        testSet['label'][i] = np.array([0,0,1])


print(testSet)

                           test_path      label
0    /content/folder/test/3/3_72.csv  [0, 0, 1]
1    /content/folder/test/3/3_85.csv  [0, 0, 1]
2    /content/folder/test/3/3_14.csv  [0, 0, 1]
3    /content/folder/test/3/3_91.csv  [0, 0, 1]
4    /content/folder/test/3/3_28.csv  [0, 0, 1]
..                               ...        ...
297  /content/folder/test/2/2_96.csv  [0, 1, 0]
298  /content/folder/test/2/2_99.csv  [0, 1, 0]
299  /content/folder/test/2/2_56.csv  [0, 1, 0]
300  /content/folder/test/2/2_61.csv  [0, 1, 0]
301  /content/folder/test/2/2_39.csv  [0, 1, 0]

[302 rows x 2 columns]


##Define CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, Data_frame):
        self.train_path_list = Data_frame["train_path"]
        self.label_list = Data_frame["label"]

    def __getitem__(self, index):
        train_path = self.train_path_list[index]
        df = pd.read_csv(train_path, usecols=[1], header=None)
        a = np.array(df[1:201], dtype = np.float32)
        b = np.transpose(a)
        td = torch.tensor(b, dtype=torch.float)
        td = F.interpolate(td.unsqueeze(0), size=(1024)).squeeze(0)
        label_ = self.label_list[index]

        tmp = {"train_data" : td  , "labels" : torch.tensor(label_)}

        return tmp

    def __len__(self):
        return len(self.train_path_list)

class TestDataset(Dataset):
    def __init__(self, Data_frame):
        self.test_path_list = Data_frame["test_path"]
        self.label_list = Data_frame["label"]

    def __getitem__(self, index):
        test_path = self.test_path_list[index]
        df = pd.read_csv(test_path, usecols=[1], header=None)
        a = np.array(df[1:201], dtype = np.float32)
        b = np.transpose(a)
        td = torch.tensor(b, dtype=torch.float)
        td = F.interpolate(td.unsqueeze(0), size=(1024)).squeeze(0)
        label_ = self.label_list[index]

        tmp = {"test_data" : td  , "labels" : torch.tensor(label_)}

        return tmp

    def __len__(self):
        return len(self.test_path_list)

##Dataloader

In [ ]:
batch_size = 1

train_dataset = CustomDataset(trainSet)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1,
                                          shuffle=True, num_workers=1)


test_dataset = TestDataset(testSet)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                          shuffle=False, num_workers=1)

classes = ([1,0,0], [0,1,0], [0,0,1])

In [ ]:
print(len(trainloader)) #train n(img)//batch_size
print(len(testloader))  #test n(image)

300
302


## Model

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # 첫 번째 합성곱 레이어
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)

        # 두 번째 합성곱 레이어
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        # 스킵 연결을 위한 합성곱 레이어 (stride가 1보다 클 때만 사용)
        self.skip_layer = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        # 첫 번째 합성곱 레이어
        out = F.relu(self.bn1(self.conv1(x)))

        # 두 번째 합성곱 레이어
        out = self.bn2(self.conv2(out))

        # 스킵 연결
        skip_out = self.skip_layer(x) if self.skip_layer is not None else x
        out += skip_out

        # ReLU 활성화 함수
        out = F.relu(out)
        return out

class ResNet1D(nn.Module):
    def __init__(self, input_channels, num_classes=3):
        super(ResNet1D, self).__init__()

        # 초기 합성곱 레이어
        self.initial_conv = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.initial_bn = nn.BatchNorm1d(64)
        self.initial_relu = nn.ReLU(inplace=True)

        # Residual 블록
        self.layer1 = self.make_layer(64, 64, stride=2)
        self.layer2 = self.make_layer(64, 128, stride=2)

        # Global Average Pooling
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)

        # 완전 연결 레이어
        self.fc = nn.Linear(128, num_classes)

    def make_layer(self, in_channels, out_channels, stride=1):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        layers.append(ResidualBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        # 초기 합성곱 레이어
        x = self.initial_relu(self.initial_bn(self.initial_conv(x)))

        # Residual 블록
        x = self.layer1(x)
        x = self.layer2(x)

        # Global Average Pooling
        x = self.global_avg_pooling(x)

        # 완전 연결 레이어
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# 모델 생성
input_channels = 1  # 입력 데이터의 채널 수
num_classes = 3  # 분류할 클래스 수
model = ResNet1D(input_channels, num_classes)

# 모델 확인
print(model)

ResNet1D(
  (initial_conv): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (initial_bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (initial_relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_layer): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,

In [ ]:
torchsummary.summary(model,(1,1024)).float

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
model = ResNet1D(1, 3)

learning_rate=0.0001
optimizer = Adam(params=model.parameters(), lr=learning_rate)

m = nn.Softmax(dim=1) #for resnet

In [ ]:
for epoch in range(4000):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        tr , labels = data["train_data"], data["labels"]
        tr = tr.to('cuda:0' if torch.cuda.is_available() else "cpu")
        labels = labels.to('cuda:0' if torch.cuda.is_available() else "cpu").float()
        optimizer.zero_grad()

        # print(tr.size())
        outputs = m(model(tr)).view(1,3).float()

        loss = nn.CrossEntropyLoss()
        output = loss(outputs,labels)
        if i % 300 == 1:
            print("Epoch",epoch,":",output)

        output.backward()
        optimizer.step()

print('Finished Training')

Epoch 0 : tensor(0.5514, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 1 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 2 : tensor(0.5525, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 3 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 4 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 5 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 6 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 7 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 8 : tensor(0.5718, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 9 : tensor(0.5516, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 10 : tensor(0.5604, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 11 : tensor(0.5516, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 12 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 13 : tensor(0.5515, device='cuda:0', grad_fn=<DivBackward1>)
Epoch 14 : tensor(0.5532, device='cuda:0', grad_fn=<DivBackward1>)
Epoch

KeyboardInterrupt: 

In [ ]:
tttt = []

for i, data in enumerate(testloader, 0):
      trr , labelss = data["test_data"], data["labels"]
      trr = trr.to('cuda:0' if torch.cuda.is_available() else "cpu")
      labelss = labelss.to('cuda:0' if torch.cuda.is_available() else "cpu").float()

      outputss = m(model(trr)).view(1,3).float()
      print(outputss)

      tttt.append(torch.argmax(outputss,dim=1).item())




print('Finished Training')

tensor([[1.9855e-07, 9.0710e-06, 9.9999e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[3.1280e-07, 2.7723e-05, 9.9997e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[2.6284e-06, 1.8521e-04, 9.9981e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[2.9403e-07, 2.3654e-05, 9.9998e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[9.5288e-07, 1.4655e-05, 9.9998e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[4.8193e-04, 9.9826e-01, 1.2582e-03]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[3.0119e-06, 1.7512e-03, 9.9825e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[1.8056e-07, 1.3353e-05, 9.9999e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[2.1598e-05, 5.9535e-03, 9.9402e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[2.1508e-06, 5.6733e-03, 9.9432e-01]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[1.1456e-07, 3.8992e-05, 9.9996e

In [ ]:
A = torch.argmax(labelss,dim=1)-torch.argmax(outputss,dim=1)
# print(A)
print(torch.count_nonzero(A, dim=0))

torch.argmax(outputss,dim=1)

# torch.argmax(labelss,dim=1)

tensor(0, device='cuda:0')


tensor([1], device='cuda:0')

In [ ]:
torch.save(model, 'best.pth')

In [ ]:
print(tttt)

[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1]
